In [31]:
from bs4 import BeautifulSoup
import requests as rq
import re
import datetime
import pymysql
import brotli
from flask import Flask,jsonify, request
from fake_useragent import UserAgent
import cv2
import numpy as np
import time
from urllib.parse import urlencode,urlsplit,parse_qsl
import threading
import sys
import os
import json
import shutil
import traceback
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import base64
os.environ['REQUESTS_CA_BUNDLE'] =  os.path.join(os.path.dirname(sys.argv[0]), 'cacert.pem')
print(os.path.join(os.path.dirname(sys.argv[0]), 'cacert.pem'))
import telnetlib
import configparser

D:\Program Files\anaconda3\envs\govpur\lib\site-packages\cacert.pem


In [32]:
db_settings = {
    "host": '60.250.109.71',
    "port": 23306,
    "user": 'xuan',
    "password": 'Qaz123',
    "db": 'tender',
    "charset": 'utf8mb4',
    "autocommit":True
}

In [40]:
#取出字典某 key 值對應 value，若找無此 key 值，則回傳空字串 
def dicMemberCheck(dicObj, key):
    if key in dicObj:
        return dicObj[key]
    else:
        return ""
# dicMemberCheck('標案案號', {'標案案號':123,'ji3':456})

In [70]:
count=1000
command_update=f"update {db_settings['db']}.gov_purchase_new set dataPK=%s where proposer_name=%s and bid_no=%s and bid_name=%s and times=%s and get_data_date=%s and tenderType=%s"
conn = pymysql.connect(**db_settings)
# 建立Cursor物件
with conn.cursor(pymysql.cursors.DictCursor) as cursor:
    command=f"select proposer_name,bid_no,bid_name,times,get_data_date,tenderType,detail_connect FROM {db_settings['db']}.gov_purchase_new where dataPK is null limit 0,{count}"
    cursor.execute(command)
    datas = cursor.fetchall()
    print("首次抓取資料")
    print(datetime.datetime.now())
    print(len(datas))
    while len(datas)==count:
        data_combine=[]
        for i in datas:
            pk=''
            dic = dict(parse_qsl(urlsplit(i['detail_connect']).query))
            if i['tenderType'] == 'final':
                if dicMemberCheck(dic,'pk')!="":
                    pk = dicMemberCheck(dic,'pk')
                elif dicMemberCheck(dic,'pkAtmMain')!="":
                    pk = dicMemberCheck(dic,'pkAtmMain')
            elif i['tenderType'] == 'tenderDeclaration':
                if dicMemberCheck(dic,'pk')!="":
                    pk = dicMemberCheck(dic,'pk')
                elif dicMemberCheck(dic,'primaryKey')!="":
                    pk = dicMemberCheck(dic,'primaryKey')
            elif i['tenderType'] == 'searchAppeal':
                url = 'http://localhost:8888?' + i['detail_connect']
                dic = dict(parse_qsl(urlsplit(url).query))
                if dicMemberCheck(dic,'pkTA')!="":
                    pk = dicMemberCheck(dic,'pkTA')
                elif dicMemberCheck(dic,'pkTpAppealHis')!="":
                    pk = dicMemberCheck(dic,'pkTpAppealHis')
            elif i['tenderType'] == 'publicRead':
                if dicMemberCheck(dic,'pkTpRead')!="":
                    pk = dicMemberCheck(dic,'pkTpRead')
                elif dicMemberCheck(dic,'tpReadSeq')!="":
                    pk = dicMemberCheck(dic,'tpReadSeq')
            pk = pk + i['tenderType'][0]
            if pk=='':
                print(i)
            else:
                data_combine.append((pk,i['proposer_name'],i['bid_no'],i['bid_name'],i['times'],str(i['get_data_date']),i['tenderType']))
        print("資料組合完成")
        print(datetime.datetime.now())
        cursor.executemany(command_update, data_combine)
        print("重新抓取資料")
        print(datetime.datetime.now())
        command=f"select proposer_name,bid_no,bid_name,times,get_data_date,tenderType,detail_connect FROM {db_settings['db']}.gov_purchase_new where dataPK is null limit 0,{count}"
        cursor.execute(command)
        datas = cursor.fetchall()

首次抓取資料
2022-07-05 12:47:04.475130
1000
資料組合完成
2022-07-05 12:47:04.489143
重新抓取資料
2022-07-05 12:48:02.146966
資料組合完成
2022-07-05 12:48:03.960959
重新抓取資料
2022-07-05 12:48:59.909967
資料組合完成
2022-07-05 12:49:01.474333
重新抓取資料
2022-07-05 12:50:00.447843
資料組合完成
2022-07-05 12:50:02.568873
重新抓取資料
2022-07-05 12:50:56.422616
資料組合完成
2022-07-05 12:50:58.270061
重新抓取資料
2022-07-05 12:51:52.493150
資料組合完成
2022-07-05 12:51:54.368748
重新抓取資料
2022-07-05 12:52:59.815266
資料組合完成
2022-07-05 12:53:01.341361
重新抓取資料
2022-07-05 12:53:56.204190
資料組合完成
2022-07-05 12:53:57.976189
重新抓取資料
2022-07-05 12:55:06.804658
資料組合完成
2022-07-05 12:55:08.647856
重新抓取資料
2022-07-05 12:56:06.488624
資料組合完成
2022-07-05 12:56:08.463980
重新抓取資料
2022-07-05 12:57:15.639299
資料組合完成
2022-07-05 12:57:17.649019
重新抓取資料
2022-07-05 12:58:12.837519
資料組合完成
2022-07-05 12:58:15.254120
重新抓取資料
2022-07-05 12:59:17.500932
資料組合完成
2022-07-05 12:59:19.972282
重新抓取資料
2022-07-05 13:00:14.408457
資料組合完成
2022-07-05 13:00:17.073483
重新抓取資料
2022-07-05 13:01:08.958849
資料組合完成
20

資料組合完成
2022-07-05 14:53:41.215130
重新抓取資料
2022-07-05 14:54:39.652471
資料組合完成
2022-07-05 14:54:41.953443
重新抓取資料
2022-07-05 14:55:34.312879
資料組合完成
2022-07-05 14:55:36.537846
重新抓取資料
2022-07-05 14:56:24.785775
資料組合完成
2022-07-05 14:56:26.735072
重新抓取資料
2022-07-05 14:57:12.242542
資料組合完成
2022-07-05 14:57:14.672944
重新抓取資料
2022-07-05 14:58:04.304468
資料組合完成
2022-07-05 14:58:06.238564
重新抓取資料
2022-07-05 14:58:56.102116
資料組合完成
2022-07-05 14:58:57.978661
重新抓取資料
2022-07-05 14:59:47.557950
資料組合完成
2022-07-05 14:59:49.820890
重新抓取資料
2022-07-05 15:00:43.200456
資料組合完成
2022-07-05 15:00:45.331464
重新抓取資料
2022-07-05 15:01:35.214934
資料組合完成
2022-07-05 15:01:36.906194
重新抓取資料
2022-07-05 15:02:25.002560
資料組合完成
2022-07-05 15:02:26.774416
重新抓取資料
2022-07-05 15:03:12.592524
資料組合完成
2022-07-05 15:03:14.592297
重新抓取資料
2022-07-05 15:04:02.458512
資料組合完成
2022-07-05 15:04:05.194535
重新抓取資料
2022-07-05 15:05:27.674602
資料組合完成
2022-07-05 15:05:29.783118
重新抓取資料
2022-07-05 15:06:17.131459
資料組合完成
2022-07-05 15:06:19.028161
重新抓取資料
2022-07

In [68]:
command_update

'update tender.gov_purchase_new set dataPK=%s where proposer_name=%s and bid_no=%s and bid_name=%s and times=%s and get_data_date=%s and tenderType=%s'

In [66]:
data_combine

[('53671954f',
  '海洋國家公園管理處',
  '110212',
  '東吉村113號傳統建築修護工程',
  '',
  '2022-03-16 00:00:00',
  'final'),
 ('53750463t',
  '海洋委員會海巡署',
  'CGA111018',
  '研析室會議影像及麥克風系統',
  '01',
  '2022-03-16 00:00:00',
  'tenderDeclaration'),
 ('53672522f',
  '海洋委員會海巡署南部分署',
  'CGS111-012C1',
  '中芸漁港安檢所新建工程',
  '',
  '2022-03-16 00:00:00',
  'final'),
 ('53749421t',
  '海洋委員會海巡署南部分署',
  'CGS111-012C1',
  '中芸漁港安檢所新建工程',
  '02',
  '2022-03-16 00:00:00',
  'tenderDeclaration'),
 ('53672822f',
  '海洋委員會海巡署教育訓練測考中心',
  'ETTC11105L',
  '111年英語人才儲備訓練',
  '',
  '2022-03-16 00:00:00',
  'final'),
 ('53749670t',
  '海洋委員會海巡署教育訓練測考中心',
  'ETTC11105L',
  '111年英語人才儲備訓練',
  '02',
  '2022-03-16 00:00:00',
  'tenderDeclaration'),
 ('53671578f',
  '海洋委員會海巡署東南沙分署',
  'DNB11110E',
  '高雄海巡隊南沙分隊營舍新建工程',
  '',
  '2022-03-16 00:00:00',
  'final'),
 ('53748922t',
  '海洋委員會海巡署東南沙分署',
  'DNB11110E',
  '高雄海巡隊南沙分隊營舍新建工程',
  '02',
  '2022-03-16 00:00:00',
  'tenderDeclaration'),
 ('53672811f',
  '海洋委員會海巡署艦隊分署',
  'B110238',
  '高緯度遠洋巡護